In [ ]:
#!pip install pytesseract 
#!pip install opencv 
#!pip install numpy as np
#!pip install pillow
#!pip install postal

In [1]:
import pytesseract 
from PIL import Image, ImageOps
import numpy as np
# from postal.parser import parse_address

In [152]:

def extract_text(url):
    import pytesseract 
    from PIL import Image
    
    """extracts test out of image
    --inputs= image_url """
    #opening the image
    im=Image.open(url)
    grey= im.convert(mode="L")
    #threshold= 160
    #image=grey.point(lambda x: 255 if x > threshold else 0)
    
    #configuration for pytesseract to operate with text extraction
    config = ('-l eng --oem 1 --psm 3')
    text = pytesseract.image_to_string(grey, config=config)
    
    return text

In [153]:
#extract address --in progress.... 
def extract_address(text):
    import re
    text=text
    regexp = "[0-9]{1,3} .+, .+, [A-Z]{2} [0-9]{5}"
    address = re.findall(regexp, text)
    return address 


#extract order

#def order():
    
#extract total

#def extract_total():
    
#makesure to have the total mapped into numbers if not extracted as a number, try to map the letters into numbers.





In [154]:

image=(r"images/large-receipt-image-dataset-SRD/1012-receipt.jpg")
text=extract_text(image)
print (text)

#14

HP Pho Ga
8930 Mission Dr. #102
pp ooee 91770
Phone (626)268-9999

 
     

bate: Apr 01, 2019 Time; 05:

Server: Admin

Bill; 59980 Table : 14

1 Pho Ga (Small) 8.00

1 Pho Ga (Large) 9.00

1 Gor Cuon 5.25
“Subtotal 22.25
TAX 211
Total 24.36
8. Service Gharge- 40% 8.90

Total $33 26

Suggested Tip : 15% (3,65)

18% (4.38)

20% (4.87)
Open Time : Apr 01, 2019 04:39PM
Printed By : Admin

Thank you for Coming !



# use this !!!!

In [12]:
import cv2
import numpy as np
import utlis
 
 
########################################################################
webCamFeed = True
pathImage = "1.jpg"
cap = cv2.VideoCapture(0)
cap.set(10,160)
heightImg = 640
widthImg  = 480
########################################################################
 
utlis.initializeTrackbars()
count=0
 
while True:
 
    if webCamFeed:success, img = cap.read()
    else:img = cv2.imread(pathImage)
    img = cv2.resize(img, (widthImg, heightImg)) # RESIZE IMAGE
    imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8) # CREATE A BLANK IMAGE FOR TESTING DEBUGING IF REQUIRED
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # CONVERT IMAGE TO GRAY SCALE
    imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # ADD GAUSSIAN BLUR
    thres=utlis.valTrackbars() # GET TRACK BAR VALUES FOR THRESHOLDS
    imgThreshold = cv2.Canny(imgBlur,thres[0],thres[1]) # APPLY CANNY BLUR
    kernel = np.ones((5, 5))
    imgDial = cv2.dilate(imgThreshold, kernel, iterations=2) # APPLY DILATION
    imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # APPLY EROSION
 
    ## FIND ALL COUNTOURS
    imgContours = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
    imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
    contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # FIND ALL CONTOURS
    cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10) # DRAW ALL DETECTED CONTOURS
 
 
    # FIND THE BIGGEST COUNTOUR
    biggest, maxArea = utlis.biggestContour(contours) # FIND THE BIGGEST CONTOUR
    if biggest.size != 0:
        biggest=utlis.reorder(biggest)
        cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20) # DRAW THE BIGGEST CONTOUR
        imgBigContour = utlis.drawRectangle(imgBigContour,biggest,2)
        pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
        pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
 
        #REMOVE 20 PIXELS FORM EACH SIDE
        imgWarpColored=imgWarpColored[20:imgWarpColored.shape[0] - 20, 20:imgWarpColored.shape[1] - 20]
        imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))
 
        # APPLY ADAPTIVE THRESHOLD
        imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
        imgAdaptiveThre= cv2.adaptiveThreshold(imgWarpGray, 255, 1, 1, 7, 2)
        imgAdaptiveThre = cv2.bitwise_not(imgAdaptiveThre)
        imgAdaptiveThre=cv2.medianBlur(imgAdaptiveThre,3)
 
        # Image Array for Display
        imageArray = ([img,imgGray,imgThreshold,imgContours],
                      [imgBigContour,imgWarpColored, imgWarpGray,imgAdaptiveThre])
 
    else:
        imageArray = ([img,imgGray,imgThreshold,imgContours],
                      [imgBlank, imgBlank, imgBlank, imgBlank])
 
    # LABELS FOR DISPLAY
    lables = [["Original","Gray","Threshold","Contours"],
              ["Biggest Contour","Warp Prespective","Warp Gray","Adaptive Threshold"]]
 
    stackedImage = utlis.stackImages(imageArray,0.75,lables)
    cv2.imshow("Result",stackedImage)
 
    # SAVE IMAGE WHEN 's' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("Scanned/myImage"+str(count)+".jpg",imgWarpColored)
        item_name=("Scanned/myImage"+str(count)+".jpg")
        cv2.rectangle(stackedImage, ((int(stackedImage.shape[1] / 2) - 230), int(stackedImage.shape[0] / 2) + 50),
                      (1100, 350), (0, 255, 0), cv2.FILLED)
        cv2.putText(stackedImage, "Scan Saved", (int(stackedImage.shape[1] / 2) - 200, int(stackedImage.shape[0] / 2)),
                    cv2.FONT_HERSHEY_DUPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        cv2.imshow('Result', stackedImage)
        cv2.waitKey(300)
        count += 1

        

KeyboardInterrupt: 

# Image cleaning and cropping function
-- this function performs the following transformations 
---- resize to 640,480 
---- BGR to grey
---- Apply GaussianBlur (5,5),1
---- Uses adaptive threshold (guissan Binary) 
---- dialate the image 
---- erode 
---- find contours 
---- apply contours 
---- 

In [19]:


def clean_image(pathImage):
    import cv2
    import numpy as np
    import utlis

    #cap.set(10,160)
    heightImg = 1920
    widthImg  = 1080

    
    #pathImage = r"images/large-receipt-image-dataset-SRD/1013-receipt.jpg"

    img = cv2.imread(pathImage)
    img = cv2.resize(img, (widthImg, heightImg)) # RESIZE IMAGE
    imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8) # CREATE A BLANK IMAGE FOR TESTING DEBUGING IF REQUIRED
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # CONVERT IMAGE TO GRAY SCALE
    imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # ADD GAUSSIAN BLUR
    #thres=utlis.valTrackbars() # GET TRACK BAR VALUES FOR THRESHOLDS
    imgThreshold = cv2.adaptiveThreshold(imgBlur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,11,2) # APPLY CANNY BLUR
    kernel = np.ones((5, 5))
    imgDial = cv2.dilate(imgThreshold, kernel, iterations=2) # APPLY DILATION
    imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # APPLY EROSION

        ## FIND ALL COUNTOURS
    imgContours = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
    imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
    contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # FIND ALL CONTOURS
    cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10) # DRAW ALL DETECTED CONTOURS


        # FIND THE BIGGEST COUNTOUR
    biggest, maxArea = utlis.biggestContour(contours) # FIND THE BIGGEST CONTOUR
    if biggest.size != 0:
            biggest=utlis.reorder(biggest)
            cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20) # DRAW THE BIGGEST CONTOUR
            imgBigContour = utlis.drawRectangle(imgBigContour,biggest,2)
            pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
            pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

            #REMOVE 20 PIXELS FORM EACH SIDE
            imgWarpColored=imgWarpColored[20:imgWarpColored.shape[0] - 20, 20:imgWarpColored.shape[1] - 20]
            imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))
           
    count=1
    cv2.imwrite("Scanned/myImage"+str(count)+".jpg",imgWarpColored)
    item_name=("Scanned/myImage"+str(count)+".jpg")
    return item_name



In [49]:
item=clean_image(r"images/large-receipt-image-dataset-SRD/1084-receipt.jpg")
import functions 
print (functions.extract_text(item))

Trattoria Sort

328 Washingtonstreet
Hoboken, Nd
201-533-1801

TABLE # 4
CHECK# 27273

ae ae 9:93:48 PM
e

SERVER: Gaston
STATION: 01

Item Count: 4

1 STUFFED nee

1 BK FETT AL PORCINI *
1 COSTOLETTE/ALLA/PAP#
1 BROCCOLI RABE W/ GAM

| ESPRESSO
1 DBLD ESPRESSO

Sot
GRAND TOT

Opened: 7/1/2018 5:53:48 PM

$117.7
8,24

"
‘10.



In [50]:
item5=functions.clean_image(r"images/large-receipt-image-dataset-SRD/1084-receipt.jpg")
print(functions.extract_text(item5))

lrattora gar

328 Washingtonstreet
oboken, Nu
101-533-1801

TABLE # 4
CHECK# 27273

ae a 7/1/2018 5:53:48 PM
SERVER; Gastone
STATION: 01

Iten Count: 4

| STUFFED eer $18.00

| BK FETT AL PORCINI * $36.00
CSOT EE ae $42 .00

| BROCCOLI RABE W/ GA $10.50
{ ESPRESSOX $3.75
1 DBLD ESPRESSO = $7.90
subtotal = $117.75

$824

0 ‘ne

Opened: 7/1/2018 5:53:46 PM

 

